# ランダムフォレストの実装

このノートブックでは、Leo Breiman氏の論文「Random Forests」のアイデアに基づき、NumPyとPython標準ライブラリのみを使用してランダムフォレスト分類器をスクラッチから実装します。

**主なステップ:**
1. 必要なライブラリと以前に作成した決定木クラスのインポート
2. データセットの準備
3. ランダムフォレストのアルゴリズム概要
4. ブートストラップサンプリングの実装
5. RandomForestClassifierクラスの実装
    - 個々の木の学習 (特徴量のランダム選択を含む)
    - 予測 (多数決)
6. モデルの学習と評価`

## 1. ライブラリと決定木クラスのインポート

In [ ]:
import numpy as np
from collections import Counter
import random
import sys, os

# 作成した決定木クラス
sys.path.append(os.path.abspath('../src/Tree'))
from decision_tree import DecisionTreeClassification, Node

## 2. データセットの準備

In [2]:
# サンプルデータ
# 特徴量: X_sample
# 列0: 数値特徴量
# 列1: カテゴリ特徴量 (0: 'A', 1: 'B', 2: 'C' と仮定)
X_sample = np.array([
    [1, 0], [2, 1], [3, 0], [4, 2], [5, 1],
    [6, 2], [7, 0], [8, 0], [9, 1], [10, 2],
    [1.5, 0], [2.5, 1], [3.5, 0], [4.5, 2], [5.5, 1],
    [6.5, 2], [7.5, 0], [8.5, 0], [9.5, 1], [10.5, 2]
])
# クラスラベル: y_sample
y_sample = np.array([0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1])

# 特徴量のタイプ
feature_types_sample = ['numeric', 'categorical']
# カテゴリ特徴量のマッピング (表示用)
category_mapping_sample = {0: 'A', 1: 'B', 2: 'C'}

print("サンプルデータ X_sample (shape):", X_sample.shape)
print("サンプルラベル y_sample (shape):", y_sample.shape)

サンプルデータ X_sample (shape): (20, 2)
サンプルラベル y_sample (shape): (20,)


## 3. ランダムフォレストのアルゴリズム概要

Breiman (2001) の論文で説明されているランダムフォレストの基本的なアルゴリズムは以下の通りです。

1.  **N個の決定木を構築する (Nはハイパーパラメータ `n_estimators`)。**  
    各木を構築するために：  
    a.  元の訓練データから、**ブートストラップサンプル**（復元抽出による同じサイズのサブサンプル）を作成する。  
    b.  このブートストラップサンプルを使って決定木を成長させる。各ノードで分割を行う際：  
           i.   全特徴量の中から、**ランダムにm個の特徴量を選択**する（mはハイパーパラメータ `max_features`）。  
           ii.  選択されたm個の特徴量の中から、最適な分割（例：Gini不純度を最小化する分割）を見つける。  
    c.  個々の木は、**枝刈りせずに**最大限に成長させる（または、`max_depth`, `min_samples_leaf`などの停止条件まで）。  

2.  **予測を行う。**  
    *   **分類の場合**: 新しいデータポイントに対して、フォレスト内の各木に予測させる。最終的な予測は、各木の予測の**多数決**によって決定する。  
    *   **回帰の場合**: 各木の予測の**平均値**を最終的な予測とする。

**ランダムフォレストの主な利点:**  
*   高い予測精度。  
*   過学習しにくい（個々の木は過学習する可能性があるが、アンサンブルすることで緩和される）。  
*   特徴量の重要度を評価できる。  
*   大規模なデータセットや高次元データにも対応可能。  
*   並列処理が容易。


## 4. ブートストラップサンプリングの実装


*   **ステップ 4.1: ブートストラップサンプリングの概念説明**
    *   ランダムフォレストでは、各決定木を学習させるために、元の訓練データから少しずつ異なるデータセットを作成します。この手法の一つがブートストラップサンプリングです。
    *   ブートストラップサンプリングとは、元のデータセット（サンプル数N）から、**復元抽出**（一度選んだサンプルを元に戻してから再度選ぶ）によってN個のサンプルを選び出し、新しいデータセットを作成する手法です。
    *   この操作を繰り返すことで、複数の異なる（しかし元のデータに基づいた）訓練サブセットが得られます。各決定木は、これらの異なるサブセットで学習されるため、木々の間に多様性が生まれます。これがランダムフォレストの過学習抑制と汎化性能向上に寄与します。
    *   平均して、ブートストラップサンプルには元のデータの約63.2%のユニークなサンプルが含まれ、残りの約36.8%のサンプルは含まれません（これらはOut-of-Bagサンプルと呼ばれ、極限を計算することで簡単に求められます。）

*   **ステップ 4.2: `bootstrap_sample` 関数の実装**
    *   **目的**: 与えられた特徴量データ `X` とラベルデータ `y` から、1つのブートストラップサンプルを生成します。
    *   **処理の流れ**:
        1.  元のデータセットのサンプル数を取得します。
        2.  `np.random.choice` を使用して、0から (サンプル数-1) までのインデックスを、サンプル数と同じ個数だけ**復元抽出**します (`replace=True`)。
        3.  得られたインデックスを使って、元の `X` と `y` から新しいデータセット `X_bootstrap` と `y_bootstrap` を作成して返します

In [3]:
def boostrap_sample(X,y) -> tuple:
    '''
    ブートストラップサンプリングを行う関数
    Parameters:
        X (np.ndarray): 特徴量データ
        y (np.ndarray): クラスラベル
    Returns:
        tuple: ブートストラップサンプルの特徴量とラベル
    '''

    n_samples = X.shape[0]
    # 復元抽出でインデックスを選択
    indices = np.random.choice(n_samples, size=n_samples, replace=True)
    return X[indices], y[indices]

In [4]:
# テスト
X_bs_test, y_bs_test = boostrap_sample(X_sample, y_sample)
print("元のデータ X_sample (shape):", X_sample.shape)
print("ブートストラップサンプル X_bs_test (shape):", X_bs_test.shape)

元のデータ X_sample (shape): (20, 2)
ブートストラップサンプル X_bs_test (shape): (20, 2)


## 5. RandomForestClassifierクラスの実装

*   **ステップ 5.1: `RandomForestClassifier` クラスの概念説明**
    *   このクラスは、ランダムフォレスト分類器全体を管理します。
    *   複数の決定木 (`DecisionTree` クラスのインスタンス) を保持し、それらの学習と予測の取りまとめを行います。
    *   **主な機能**:
        *   `__init__`: ハイパーパラメータ（木の数、各木の深さ制限など）を初期化します。
        *   `fit`: 訓練データを受け取り、指定された数の決定木をブートストラップサンプルと特徴量のランダム選択を用いて学習させます。
        *   `predict`: 新しいデータに対して、フォレスト内の全ての木に予測を行わせ、その結果を多数決で集約して最終的な予測クラスを返します。
        *   `predict_proba`: 各クラスに属する確率を予測します（各木の投票の割合）。

*   **ステップ 5.2: `RandomForestClassifier` クラスの `__init__` メソッドの実装**
    *   **目的**: ランダムフォレストのハイパーパラメータを初期化し、学習済みモデル（決定木のリストなど）を格納するための変数を準備します。
    *   **引数**:
        *   `n_estimators`: フォレストを構成する決定木の数。
        *   `max_depth`: 個々の決定木の最大深さ。`None` の場合は制限なし。
        *   `min_samples_split`: ノードを分割するために必要な最小サンプル数。
        *   `min_samples_leaf`: 葉ノードを形成するために必要な最小サンプル数。
        *   `max_features`: 各ノードで分割を検討する際にランダムに選択する特徴量の数。文字列（'sqrt', 'log2'）、整数、または浮動小数点数（割合）で指定可能。
        *   `random_state`: 乱数生成のシード。再現性を確保するために使用。
    *   **初期化する主なインスタンス変数**:
        *   `self.trees`: 学習済みの個々の決定木を格納するリスト。
        *   `self.feature_types`: `fit` メソッドで渡される特徴量の型情報。
        *   `self.classes_`: `fit` メソッドで学習データから取得するユニークなクラスラベルのリスト。`predict_proba` などで使用。

*   **ステップ 5.3: `RandomForestClassifier` クラスの `fit` メソッドの実装**
    *   **目的**: 訓練データ `X_train`, `y_train` と特徴量の型情報 `feature_types_list` を用いて、ランダムフォレストを構成する複数の決定木を学習させます。
    *   **処理の流れ**:
        1.  `random_state` が設定されていれば、乱数シードを固定します。
        2.  学習済みの木を格納する `self.trees` リストを初期化します。
        3.  訓練データからユニークなクラスラベルを取得し、`self.classes_` に保存します。
        4.  指定された `self.n_estimators` の数だけ以下のループを実行します:
            a.  `bootstrap_sample` 関数を呼び出して、現在の訓練データからブートストラップサンプル `X_bootstrap`, `y_bootstrap` を作成します。
            b.  `DecisionTree` クラスのインスタンス（個々の木）を、`max_depth` や `max_features` などのハイパーパラメータと共に初期化します。**特に `max_features` は、個々の木が分割時に特徴量をランダムに選択するために重要です。**
            c.  作成したブートストラップサンプル `X_bootstrap`, `y_bootstrap` と特徴量の型情報 `self.feature_types` を使って、個々の決定木を `fit` させます。
            d.  学習済みの木を `self.trees` リストに追加します。
        5.  学習の進捗を表示します（オプション）。

*   **ステップ 5.4: `RandomForestClassifier` クラスの `_predict_tree_outputs` ヘルパーメソッドの実装**
    *   **目的**: `predict` や `predict_proba` で共通して使用される、フォレスト内の各木からの予測結果を効率的に収集します。
    *   **処理の流れ**:
        1.  フォレストが学習済みか（`self.trees` が空でないか）を確認します。
        2.  `self.trees` リスト内の各決定木インスタンスに対して、入力されたテストデータ `X_test` の予測を `predict_single_tree` メソッド（`DecisionTree`クラスのメソッド）を呼び出して取得します。
        3.  得られた各木の予測結果をNumPy配列にまとめ、転置して返します。結果の配列の形状は `(n_samples, n_estimators)` となり、各行が1つのサンプルに対する全ツリーの予測、各列が1つのツリーによる全サンプルの予測を表します。

*   **ステップ 5.5: `RandomForestClassifier` クラスの `predict` メソッドの実装**
    *   **目的**: 新しいデータ `X_test` に対して、ランダムフォレストによる最終的なクラス予測を行います。分類タスクでは、これは通常、各木の予測の多数決によって決定されます。
    *   **処理の流れ**:
        1.  `_predict_tree_outputs` メソッドを呼び出して、`X_test` の各サンプルに対する全ツリーの予測を取得します（形状: `(n_samples, n_estimators)`）。
        2.  得られた予測結果の各行（各サンプル）に対して、最も多く出現したクラス（多数決）を計算します。`collections.Counter` を使うと効率的です。
        3.  各サンプルの多数決結果をリストに集め、NumPy配列として返します。

*   **ステップ 5.6: `RandomForestClassifier` クラスの `predict_proba` メソッドの実装**
    *   **目的**: 新しいデータ `X_test` に対して、各クラスに属する確率を予測します。これは、フォレスト内の各木がそのクラスに投票した割合として計算されます。
    *   **処理の流れ**:
        1.  `_predict_tree_outputs` メソッドを呼び出して、`X_test` の各サンプルに対する全ツリーの予測を取得します。
        2.  `fit` 時に保存した `self.classes_` を参照して、クラスの数と各クラスラベルに対応する出力配列のインデックスを決定します。
        3.  各サンプルについて、全ツリーの予測の中で各クラスが何回出現したかをカウントします。
        4.  各クラスの出現回数を木の総数 `self.n_estimators` で割ることで、そのクラスに属する確率を計算します。
        5.  結果を形状 `(n_samples, n_classes)` のNumPy配列として返します。

In [7]:
class RandomForestClassifier:
    def __init__(
            self,
            n_estimators=10,
            max_depth=None,
            min_samples_split=2,
            min_samples_leaf=1,
            max_features='sqrt',
            random_state=None
    ):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.random_state = random_state

        self.trees = []
        self.feature_types = None
        self.classes = None

    def fit(self, X_train, y_train, feature_types_ls):
        '''
        ランダムフォレストの学習を行うメソッド
        Parameters:
            X_train (np.ndarray): 学習用特徴量データ
            y_train (np.ndarray): 学習用クラスラベル
            feature_types_ls (list): 特徴量のタイプリスト
        '''
        
        if self.random_state is not None:
            np.random.seed(self.random_state)
            random.seed(self.random_state)

        self.trees = []
        self.feature_types = feature_types_ls
        self.classes = np.unique(y_train)
        n_samples, n_features = X_train.shape

        print("ランダムフォレストの学習を開始します...")

        for i in range(self.n_estimators):
            # 1. ブートストラップサンプリング
            X_bs, y_bs = boostrap_sample(X_train, y_train)

            # 2. 個々の決定木を初期化して学習
            tree = DecisionTreeClassification(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                min_samples_leaf=self.min_samples_leaf,
                max_features=self.max_features,
            )

            tree.fit(X_bs, y_bs, self.feature_types)
            self.trees.append(tree)
            
            # 3. 学習の進捗を表示
            progress_interval = self.n_estimators // 10 if self.n_estimators >= 10 else 1

            if (i + 1) % progress_interval == 0 or i == self.n_estimators - 1:
                print(f"決定木 {i + 1}/{self.n_estimators} の学習が完了しました。")
            
        print("ランダムフォレストの学習が完了しました。")

    def predict_tree_outputs(self, X_test):
        '''
        フォレスト内の各木からの予測を取得するヘルパー関数。
        Returns:
            np.array: 各サンプルに対する各木の予測 (n_samples, n_estimators)
        '''
        
        if not self.trees:
            raise ValueError("モデルが学習されていません。`fit`メソッドを呼び出してください。")
        
        # DecisionTreeクラスのpredictがpredict_single_treeメソッドに変更されているので注意
        tree_predictions = np.array([tree.predict_single_tree(X_test) for tree in self.trees])

        # (n_estimators, n_samples) の形状にするために転地
        return tree_predictions.T
    
    def predict(self, X_test):
        '''
        多数決で予測を行うメソッド
        '''
        tree_preds = self.predict_tree_outputs(X_test) # (n_samples, n_estimators)

        forest_preds = []

        for sample_preds in tree_preds: # sample_predsは各サンプルに対する全ての木の予測
            counts = Counter(sample_preds)
            majority_vote = counts.most_common(1)[0][0]  # 最も頻出のクラスを取得
            forest_preds.append(majority_vote)

        return np.array(forest_preds)
    
    def predict_proba(self, X_test):
        '''
        新しいデータポイントの各クラスの確率を予測するメソッド
        '''

        tree_preds = self.predict_tree_outputs(X_test) # (n_samples, n_estimators)
        
        n_samples = X_test.shape[0]
        if self.classes is None:
            raise ValueError("モデルが学習されていません。`fit`メソッドを呼び出してください。")
        num_classes = len(self.classes)
        
        # self.classesはfit時にソートされているので
        # self.classesを使って，出力確率配列の列インデックスとクラスラベルを対応させる
        class_to_index = {cls: idx for idx, cls in enumerate(self.classes)}

        # 確率を格納する配列
        proba = np.zeros((n_samples, num_classes))

        for i in range(n_samples):
            sample_tree_preds = tree_preds[i, :] # i番目のサンプルに対する全ての木の予測
            counts = Counter(sample_tree_preds)

            for class_label, count in counts.items():
                # 予測されたクラスラベルがself.classesに存在する場合のみ確率を更新
                if class_label in class_to_index:
                    proba[i, class_to_index[class_label]] = count / self.n_estimators

        return proba

## 6. モデルの学習と評価

### 6.1 サンプルデータでのテスト

In [8]:
rf_classifier_sample = RandomForestClassifier(
    n_estimators=10,
    max_depth=3,
    min_samples_leaf=1,
    max_features='sqrt',
    random_state=42
)

rf_classifier_sample.fit(X_sample, y_sample, feature_types_sample)

predictions_sample = rf_classifier_sample.predict(X_sample)
print("\nSample Data - Random Forest Predictions:", predictions_sample)
print("Sample Data - True Labels:          ", y_sample)

accuracy_sample = np.sum(predictions_sample == y_sample) / len(y_sample)
print(f"Sample Data - Random Forest Accuracy: {accuracy_sample:.4f}")

proba_sample = rf_classifier_sample.predict_proba(X_sample)
print("\nSample Data - Random Forest Probabilities (first 5 samples):")
print(proba_sample[:5])

ランダムフォレストの学習を開始します...
決定木 1/10 の学習が完了しました。
決定木 2/10 の学習が完了しました。
決定木 3/10 の学習が完了しました。
決定木 4/10 の学習が完了しました。
決定木 5/10 の学習が完了しました。
決定木 6/10 の学習が完了しました。
決定木 7/10 の学習が完了しました。
決定木 8/10 の学習が完了しました。
決定木 9/10 の学習が完了しました。
決定木 10/10 の学習が完了しました。
ランダムフォレストの学習が完了しました。

Sample Data - Random Forest Predictions: [0 1 0 1 1 1 0 0 1 1 0 1 0 1 1 1 0 0 1 1]
Sample Data - True Labels:           [0 1 0 1 0 1 0 0 1 1 0 1 0 1 0 1 0 0 1 1]
Sample Data - Random Forest Accuracy: 0.9000

Sample Data - Random Forest Probabilities (first 5 samples):
[[0.9 0.1]
 [0.3 0.7]
 [1.  0. ]
 [0.1 0.9]
 [0.4 0.6]]


### 6.2 Irisデータセットでのテスト

In [9]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

iris = load_iris()
X_iris = iris.data
y_iris = iris.target
iris_feature_names = iris.feature_names
feature_types_iris = ['numeric'] * X_iris.shape[1]

X_train_iris, X_test_iris, y_train_iris, y_test_iris = train_test_split(
    X_iris, y_iris, test_size=0.3, random_state=123, stratify=y_iris
)

print("Iris Training data shape:", X_train_iris.shape)
print("Iris Test data shape:", X_test_iris.shape)

rf_classifier_iris = RandomForestClassifier(
    n_estimators=50,
    max_depth=None,
    min_samples_leaf=1,
    max_features='sqrt',
    random_state=123
)

rf_classifier_iris.fit(X_train_iris, y_train_iris, feature_types_iris)

predictions_iris_test = rf_classifier_iris.predict(X_test_iris)
accuracy_iris_test = accuracy_score(y_test_iris, predictions_iris_test)
print(f"\nIris Test Data - Random Forest Accuracy: {accuracy_iris_test:.4f}")

predictions_iris_train = rf_classifier_iris.predict(X_train_iris)
accuracy_iris_train = accuracy_score(y_train_iris, predictions_iris_train)
print(f"Iris Training Data - Random Forest Accuracy: {accuracy_iris_train:.4f}")

proba_iris_test = rf_classifier_iris.predict_proba(X_test_iris)
print("\nIris Test Data - Random Forest Probabilities (first 5 samples):")
for i in range(5):
    # iris.target_names は ['setosa', 'versicolor', 'virginica']
    # y_test_iris[i] は 0, 1, or 2
    print(f"Sample {i}: True={iris.target_names[y_test_iris[i]]}, Predicted Probs={proba_iris_test[i]}")

Iris Training data shape: (105, 4)
Iris Test data shape: (45, 4)
ランダムフォレストの学習を開始します...
決定木 5/50 の学習が完了しました。
決定木 10/50 の学習が完了しました。
決定木 15/50 の学習が完了しました。
決定木 20/50 の学習が完了しました。
決定木 25/50 の学習が完了しました。
決定木 30/50 の学習が完了しました。
決定木 35/50 の学習が完了しました。
決定木 40/50 の学習が完了しました。
決定木 45/50 の学習が完了しました。
決定木 50/50 の学習が完了しました。
ランダムフォレストの学習が完了しました。

Iris Test Data - Random Forest Accuracy: 0.9556
Iris Training Data - Random Forest Accuracy: 1.0000

Iris Test Data - Random Forest Probabilities (first 5 samples):
Sample 0: True=versicolor, Predicted Probs=[0.   0.44 0.56]
Sample 1: True=virginica, Predicted Probs=[0.   0.06 0.94]
Sample 2: True=versicolor, Predicted Probs=[0. 1. 0.]
Sample 3: True=versicolor, Predicted Probs=[0.   0.96 0.04]
Sample 4: True=virginica, Predicted Probs=[0. 0. 1.]


### 6.3 Covertypeデータセットでのテスト (大規模データセットの例)


次に、より大規模なデータセットであるCovertypeデータセットでランダムフォレストの動作を確認します。
このデータセットはサンプル数が多く、特徴量も多いため、モデルの性能と学習時間を確認するのに適しています。
注意: `fetch_covtype`は初回実行時にデータのダウンロードが発生し、時間がかかることがあります。また、全データを使用すると学習に非常に時間がかかるため、ここではデータの一部（例えば最初の50,000サンプル）を使用します。クラスラベルは1から7ですが、0から始まるように調整します。

In [21]:
from sklearn.datasets import fetch_covtype
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import time # 学習時間の計測用

# Covertypeデータセットのロード
print("Fetching Covertype dataset... (This may take a while)")
covtype = fetch_covtype()
X_covtype_full = covtype.data
y_covtype_full = covtype.target 
# クラスラベルは1から7なので、0から6に変換します
y_covtype_full = y_covtype_full - 1 
covtype_feature_names = [f"feature_{i}" for i in range(X_covtype_full.shape[1])] # 特徴量名 (仮)
# Covertypeデータセットは全て数値特徴量 (一部バイナリ含む)
feature_types_covtype = ['numeric'] * X_covtype_full.shape[1]

print("Covertype dataset loaded.")
print("Full data shape X:", X_covtype_full.shape) # (581012, 54)
print("Full data shape y:", y_covtype_full.shape) # (581012,)
print("Unique classes:", np.unique(y_covtype_full)) # 0 to 6

# データが非常に大きいため、一部をサンプリングして使用
n_samples_to_use = 50000 # 使用するサンプル数を調整可能
if X_covtype_full.shape[0] > n_samples_to_use:
    # サンプリングするインデックスをランダムに選択 (再現性のためシード固定)
    # stratifyはここでは難しいので単純にランダムサンプリング
    # もしくは、より良いサンプリング方法を検討 (例: train_test_splitで取得)
    # ここでは簡単のため、最初のn_samples_to_useを使用
    # random_indices = np.random.choice(X_covtype_full.shape[0], n_samples_to_use, replace=False)
    # X_covtype = X_covtype_full[random_indices]
    # y_covtype = y_covtype_full[random_indices]
    
    # より代表的なサンプルを得るために、層化サンプリングを試みる (元のデータが大きいので大変かも)
    # 簡単のため、まず全体を分割し、その訓練データの一部を使う
    _, X_covtype_subset, _, y_covtype_subset = train_test_split(
        X_covtype_full, y_covtype_full, test_size=n_samples_to_use/X_covtype_full.shape[0] if n_samples_to_use < X_covtype_full.shape[0] else 0.1, 
        stratify=y_covtype_full, random_state=123
    )
    if n_samples_to_use < X_covtype_full.shape[0] :
        X_covtype = X_covtype_subset
        y_covtype = y_covtype_subset
    else: # 全データ使う場合（非推奨、時間がかかる）
        X_covtype = X_covtype_full
        y_covtype = y_covtype_full

    print(f"Using a subset of {X_covtype.shape[0]} samples for Covertype test.")
else:
    X_covtype = X_covtype_full
    y_covtype = y_covtype_full
    print("Using the full (small) Covertype dataset for test.")


# データを学習用とテスト用に分割
X_train_cov, X_test_cov, y_train_cov, y_test_cov = train_test_split(
    X_covtype, y_covtype, test_size=0.3, random_state=123, stratify=y_covtype
)

print("Covertype Training data subset shape:", X_train_cov.shape)
print("Covertype Test data subset shape:", X_test_cov.shape)

# Covertypeデータセット用ランダムフォレストモデル
# 木の数や深さを調整して学習時間を考慮
rf_classifier_covtype = RandomForestClassifier(
    n_estimators=30,      # 木の数をIrisより少なめに (時間短縮のため)
    max_depth=15,         # 深さも少し制限 (時間とメモリのため)
    min_samples_leaf=5,   # 過学習を少し抑制
    max_features='sqrt',  # sqrt(54) approx 7
    random_state=123
)

print("\nStarting Covertype Random Forest training...")
start_time = time.time()

# 学習
rf_classifier_covtype.fit(X_train_cov, y_train_cov, feature_types_covtype)

end_time = time.time()
training_time = end_time - start_time
print(f"Covertype Random Forest training finished in {training_time:.2f} seconds.")

# テストデータで予測
print("\nPredicting on Covertype test data...")
start_pred_time = time.time()
predictions_covtype_test = rf_classifier_covtype.predict(X_test_cov)
end_pred_time = time.time()
prediction_time = end_pred_time - start_pred_time
print(f"Prediction finished in {prediction_time:.2f} seconds.")


# 精度評価
accuracy_covtype_test = accuracy_score(y_test_cov, predictions_covtype_test)
print(f"\nCovertype Test Data - Random Forest Accuracy: {accuracy_covtype_test:.4f}")

# (比較) 学習データでの精度
print("\nPredicting on Covertype training data (for comparison)...")
predictions_covtype_train = rf_classifier_covtype.predict(X_train_cov)
accuracy_covtype_train = accuracy_score(y_train_cov, predictions_covtype_train)
print(f"Covertype Training Data - Random Forest Accuracy: {accuracy_covtype_train:.4f}")

# (おまけ) 確率予測 (最初の数サンプルのみ)
# print("\nCovertype Test Data - Random Forest Probabilities (first 3 samples):")
# if len(X_test_cov) > 0:
#     proba_covtype_test = rf_classifier_covtype.predict_proba(X_test_cov[:3])
#     covtype_target_names = [f"Type {i+1}" for i in range(len(np.unique(y_covtype_full)))] # 仮のクラス名
#     for i in range(min(3, len(X_test_cov))):
#         true_class_idx = y_test_cov[i]
#         true_class_name = covtype_target_names[true_class_idx] if true_class_idx < len(covtype_target_names) else f"Raw_{true_class_idx}"
#         print(f"Sample {i}: True={true_class_name}, Predicted Probs={proba_covtype_test[i]}")
# else:
#     print("Test set is empty, cannot show probabilities.")

Fetching Covertype dataset... (This may take a while)
Covertype dataset loaded.
Full data shape X: (581012, 54)
Full data shape y: (581012,)
Unique classes: [0 1 2 3 4 5 6]
Using a subset of 50000 samples for Covertype test.
Covertype Training data subset shape: (35000, 54)
Covertype Test data subset shape: (15000, 54)

Starting Covertype Random Forest training...
ランダムフォレストの学習を開始します...
決定木 3/30 の学習が完了しました。
決定木 6/30 の学習が完了しました。
決定木 9/30 の学習が完了しました。
決定木 12/30 の学習が完了しました。
決定木 15/30 の学習が完了しました。
決定木 18/30 の学習が完了しました。
決定木 21/30 の学習が完了しました。
決定木 24/30 の学習が完了しました。
決定木 27/30 の学習が完了しました。
決定木 30/30 の学習が完了しました。
ランダムフォレストの学習が完了しました。
Covertype Random Forest training finished in 1475.72 seconds.

Predicting on Covertype test data...
Prediction finished in 2.01 seconds.

Covertype Test Data - Random Forest Accuracy: 0.7495

Predicting on Covertype training data (for comparison)...
Covertype Training Data - Random Forest Accuracy: 0.7710


## 7. 考察 (論文との関連など)

*   **実装したアルゴリズム**:
    *   ブートストラップサンプリング（バギング）と特徴量のランダム選択を組み合わせたアンサンブル学習。
    *   個々の決定木は、指定された `max_features` に基づいてランダムに選択された特徴量サブセットから最適な分割を選んで成長する。
    *   予測は、分類の場合は多数決。
*   **論文 (Breiman, 2001) との関連**:
    *   論文の基本的なアイデアである「複数の木を成長させ、それぞれにランダム性を導入し、それらを組み合わせる」という方針に従っている。
    *   論文で強調されている「個々の木は枝刈りしない（do not prune）」という点は、`max_depth=None` や `min_samples_leaf=1` などの設定で近似的に実現できる。
    *   特徴量のランダム選択 (Random Subspace Method の一種) は、`max_features` パラメータによって制御される。論文では `F` という記号で、各ノードでランダムに選択する入力変数の数を表している (Section 4. Random forests using random input selection)。
    *   Out-of-Bag (OOB) 推定による誤差評価や変数重要度の計算は、この基本的な実装には含まれていないが、論文では重要な要素として議論されている (Section 3.1, Section 10)。
*   **改善点や今後の課題**:
    *   **Out-of-Bag (OOB) 誤差推定の実装**: 各木を学習する際に使用されなかったサンプル（OOBサンプル）を使って、モデルの汎化性能を評価する。
    *   **特徴量重要度の計算**: OOBサンプルを使って、各特徴量が予測精度にどれだけ貢献しているかを評価する。
    *   **並列処理**: 個々の木の学習は独立して行えるため、並列化することで学習時間を大幅に短縮できる。
    *   **回帰タスクへの対応**: 現在は分類のみだが、葉ノードの値を平均値にし、アンサンブルの予測も平均値にすることで回帰に対応可能。
    *   **`predict_proba` の堅牢性向上**: `fit` 時にクラスラベルの情報を保存し（`self.classes_`として実装済み）、それを利用して確率計算の際のクラス数を正確に把握するようにした。